In [1]:
#import liberaries

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [2]:
#connect to Cassandra Cluster

def cassandra_connection(path, cassandra_config):
    cloud_config= {
      'secure_connect_bundle': path
    }
    auth_provider = PlainTextAuthProvider(cassandra_config['ASTRA_CLIENT_ID'], cassandra_config['ASTRA_CLIENT_SECRET'])
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    session = cluster.connect()
    return session

In [4]:
#create table for storing Top Trends 

def create_table(session, keyspace, table_name, column_names, column_types, primary_key):
    qry= 'CREATE TABLE IF NOT EXISTS ' + keyspace + '.' + table_name + '('
    
    for i in range(len(column_names)):
        qry = qry + column_names[i] +  ' ' + column_types[i] + ', '
    
    
    if primary_key:
        qry = qry + 'PRIMARY KEY('
        for pk in primary_key:
            qry = qry + pk + ', '
    
        qry = qry[:-2] + '));'
        
    else:
         qry = qry[:-2] + ');'
    
    
    print(qry)
    session.execute(qry)

In [5]:
def insert_into_table_from_df(session, keyspace, table_name, column_names, df):
    qry= 'INSERT INTO ' + keyspace + '.' + table_name + ' ' + column_names + ' '  + 'VALUES (%s, %s)'
    for key, value in df.items():
        print(qry, key, value)
        session.execute(qry, (key, value))

In [7]:
def select_from_table(session, keyspace, table_name):
    qry= 'SELECT * from '+ keyspace + '.' + table_name + ';'
    print(qry)
    result = session.execute(qry)
    for x in result:
        print(x[0], x[1])